In [1]:
import numpy as np
import pandas as pd
import os
import fnmatch as fn
import json
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import gensim.downloader as api
from gensim.models import TfidfModel, Word2Vec
from gensim.corpora import Dictionary
import gensim
from sklearn.preprocessing import StandardScaler
from gensim.test.utils import common_texts
from gensim.models import KeyedVectors
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss, roc_curve, auc
from imblearn.over_sampling import SMOTE
from sklearn import svm
from sklearn.metrics import accuracy_score
from gensim.models import FastText

In [8]:
def getDf(wv, words):
    for page in words['pages']:
        for block in page['blocks']:
            for annotation in block['annotations']:
                row = {
                    "text": annotation.get("text"),
                    "is_italic": annotation.get("is_italic"),
                    "x_top_left": annotation.get("x_top_left"),
                    "font_size": annotation.get("font_size"),
                    "start": annotation.get("start"),
                    "is_bold": annotation.get("is_bold"),
                    "font_name": annotation.get("font_name"),
                    "is_normal": annotation.get("is_normal"),
                    "y_top_left": annotation.get("y_top_left"),
                    "width": annotation.get("width"),
                    "end": annotation.get("end"),
                    "height": annotation.get("height"),
                    "w2v": wv[annotation.get("text")].mean(),
                }
                df.loc[len(df)] = row
                
def getDf3(wv, words):
    df = pd.DataFrame(columns=["text", "is_italic", "x_top_left", "font_size", "start", "is_bold", "font_name", "is_normal", "y_top_left", "width", "end", "height", "w2v"])
    for page in words['pages']:
        for block in page['blocks']:
            for annotation in block['annotations']:
                row = {
                    "text": annotation.get("text"),
                    "is_italic": annotation.get("is_italic"),
                    "x_top_left": annotation.get("x_top_left"),
                    "font_size": annotation.get("font_size"),
                    "start": annotation.get("start"),
                    "is_bold": annotation.get("is_bold"),
                    "font_name": annotation.get("font_name"),
                    "is_normal": annotation.get("is_normal"),
                    "y_top_left": annotation.get("y_top_left"),
                    "width": annotation.get("width"),
                    "end": annotation.get("end"),
                    "height": annotation.get("height"),
                    "w2v": wv[annotation.get("text")].mean(),
                }
                df.loc[len(df)] = row
    return df

In [12]:
def getText2(words):
    str1 = []
    for i in range(len(words['pages'])):
        for j in range (len(words['pages'][i]["blocks"])):
            for k in range (len(words['pages'][i]["blocks"][j]["annotations"])):
                str1.append(words['pages'][i]["blocks"][j]["annotations"][k]["text"])
    return str1

path = "C:\\Users\\Sheri\\IdeaProjects\\Work\\files"
listOfFiles = os.listdir(r"C:\\Users\\Sheri\\IdeaProjects\\Work\\files")
docs = []
for i in range(1,11):
    frame = path + "\\" + str(i) + ".json"
    with open(frame, "r", encoding="utf-8") as file:
        docs.append(getText2(json.load(file)))

In [13]:
from pathlib import Path  

def is_valid_url(url):
    import re
    regex = re.compile(
        r'^https?://'  # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+[A-Z]{2,6}\.?|'  # domain...
        r'localhost|'  # localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?'  # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return url is not None and regex.search(url)

model = Word2Vec(sentences=docs, vector_size=1, min_count=1, workers=4, epochs=10)
wv = model.wv

path_to_dir = path
df = pd.DataFrame(columns=["text", "is_italic", "x_top_left", "font_size", "start", "is_bold", "font_name", "is_normal", "y_top_left", "width", "end", "height", "w2v"])

for i in range(1,11):
    frame = path + "\\" + str(i) + ".json"
    with open(frame, "r", encoding="utf-8") as file:
        s = json.load(file)
        dtf = getDf3(wv, s)
        getDf(wv, s)
        filepath = Path(path + '\\' + entry + "idk" + ".csv")  
        filepath.parent.mkdir(parents=True, exist_ok=True)  
        dtf.to_csv(filepath, sep=',')  
        print(frame)

C:\Users\Sheri\IdeaProjects\Work\files\1.json
C:\Users\Sheri\IdeaProjects\Work\files\2.json
C:\Users\Sheri\IdeaProjects\Work\files\3.json
C:\Users\Sheri\IdeaProjects\Work\files\4.json
C:\Users\Sheri\IdeaProjects\Work\files\5.json
C:\Users\Sheri\IdeaProjects\Work\files\6.json
C:\Users\Sheri\IdeaProjects\Work\files\7.json
C:\Users\Sheri\IdeaProjects\Work\files\8.json
C:\Users\Sheri\IdeaProjects\Work\files\9.json
C:\Users\Sheri\IdeaProjects\Work\files\10.json


In [14]:
def get_name_keys(words):
    lst = []
    for i in range(len(words['pages'])):
        lst.append(list(words['pages'][i]["fields"].keys()))
    keys = [item for sublist in lst for item in sublist]
    return keys

def get_key_classes(keys):
    classes = {"other":0}
    counter = 1
    for key in keys:
        classes[key+"key"] = counter
        counter+=1
        classes[key+"value"] = counter
        counter+=1
    return classes

def is_intersect(x1, y1, w1, h1, x2, y2, w2, h2):
    return not (x1 + w1 < x2 or
                x2 + w2 < x1 or
                y1 + h1 < y2 or
                y2 + h2 < y1)

def getClass(row,x,name, keys_classes):
    r_top_x = row['x_top_left']
    r_top_y = row['y_top_left']
    r_width = row['width']
    r_height = row['height']
    
    key_top_x = x['key']['x_top_left']
    key_top_y = x['key']['y_top_left']
    key_width = x['key']['width']
    key_height = x['key']['height']
    
    value_top_x = x['value']['x_top_left']
    value_width = x['value']['width']
    value_top_y = x['value']['y_top_left']
    value_height = x['value']['height']
    
    if (is_intersect(r_top_x, r_top_y, r_width, r_height, key_top_x, key_top_y, key_width, key_height)):
        return keys_classes.get(name+"key")
    elif(is_intersect(r_top_x, r_top_y, r_width, r_height, value_top_x, value_top_y, value_width, value_height)):
        return keys_classes.get(name+'value')
    else:
        if 'Y' in row.index:
            if (row['Y']!= keys_classes.get("other")):
                return row['Y']
            else:
                return keys_classes.get("other")
        else:
            return keys_classes.get("other")
        
def getTrainingSet_Pr(json_str, words, keys, keys_classes):
    for i in range(len(json_str['pages'])):
        for j in keys:
            key_obj = json_str['pages'][i]['fields'][j]
            words['Y'] = words.apply(getClass, args=(key_obj, j, keys_classes), axis=1)
            
def getTrainingSet(json_str,words):
    keys = get_name_keys(json_str)
    keys_classes = get_key_classes(keys)
    getTrainingSet_Pr(json_str, words, keys, keys_classes)            

In [21]:
for i in range(1,10):
    scaler = StandardScaler()
    frame = path + "\\" + str(i) + ".jsonidk.csv"
    json_str = path + "\\" + str(i) + "t.json"
    word = pd.read_csv(frame)
    print(frame)
    with open(json_str, "r", encoding="utf-8") as f:
        template = json.load(f)
    getTrainingSet(template,word)
    X = word
    X = pd.concat([word[word['Y']!=0], word[word['Y']!=0], word[word['Y']!=0]], ignore_index=True)
    Y = X['Y']
    X = scaler.fit_transform(X.drop(columns=['text','font_name','end','start','Y', "Unnamed: 0"]), Y)
    dic = {1:20, 2:20, 3:20, 4:20, 5:20, 6:20, 7:20, 8:20, 9:20, 10:20, 11:20, 
        12:20, 13:20, 14:20}
    oversampling = SMOTE(sampling_strategy=dic, k_neighbors=2)
    X,Y = oversampling.fit_resample(X,Y)
    model = svm.SVC(kernel='rbf')
    # model = RandomForestClassifier(n_estimators=100,
    #                               random_state=42).fit(X, Y)
    model.fit(X,Y)

C:\Users\Sheri\IdeaProjects\Work\files\1.jsonidk.csv
C:\Users\Sheri\IdeaProjects\Work\files\2.jsonidk.csv
C:\Users\Sheri\IdeaProjects\Work\files\3.jsonidk.csv
C:\Users\Sheri\IdeaProjects\Work\files\4.jsonidk.csv
C:\Users\Sheri\IdeaProjects\Work\files\5.jsonidk.csv
C:\Users\Sheri\IdeaProjects\Work\files\6.jsonidk.csv
C:\Users\Sheri\IdeaProjects\Work\files\7.jsonidk.csv
C:\Users\Sheri\IdeaProjects\Work\files\8.jsonidk.csv
C:\Users\Sheri\IdeaProjects\Work\files\9.jsonidk.csv


In [22]:
word = pd.read_csv(r"C" + path[1:] + "\\10.jsonidk.csv")
temp = word
with open(r"C" + path[1:] + "\\9t.json", "r", encoding="utf-8") as f:
        template = json.load(f)
getTrainingSet(template,word)
Y_true = word['Y']
word = scaler.fit_transform(word.drop(columns=['text','font_name','end','start','Y', "Unnamed: 0"]), Y)
print(get_key_classes(get_name_keys(template)))
Y_pred = model.predict(word)
temp["Y"] = Y_pred
print(accuracy_score(Y_true,Y_pred))
t = pd.DataFrame()
t['Y'] = Y_pred
t['text'] = temp['text']
t['Y_true'] = Y_true
temp["Y_true"] = Y_true
print(t[t['Y'] != t['Y_true']][['Y', 'text', 'Y_true']])

{'other': 0, 'Номер билетаkey': 1, 'Номер билетаvalue': 2, 'Номер бронированияkey': 3, 'Номер бронированияvalue': 4, 'Пассажирkey': 5, 'Пассажирvalue': 6, 'Паспортkey': 7, 'Паспортvalue': 8, 'Рейсыkey': 9, 'Рейсыvalue': 10, 'Самолетkey': 11, 'Самолетvalue': 12, 'Багажkey': 13, 'Багажvalue': 14, 'Вылетkey': 15, 'Вылетvalue': 16, 'В путиkey': 17, 'В путиvalue': 18, 'Прибытиеkey': 19, 'Прибытиеvalue': 20}
0.5076923076923077
     Y           text  Y_true
0   16     Маршрутная       0
1   16      квитанция       0
2   10     ELECTRONIC       0
3    7         TICKET       0
4   16              /       0
5    7      ITINERARY       0
6   16        RECEIPT       0
7    6         Данные       0
8    8         полета       0
12   7          НОМЕР       1
13   7         ВАШЕГО       1
14  20        БИЛЕТА:       1
15  20     3282844214       2
16   7          НОМЕР       3
17   7  БРОНИРОВАНИЯ:       3
18   8         BT6A37       4
19  16     Информация       0
20  16              о       0
21  1

In [17]:
import eli5
from eli5.sklearn import PermutationImportance
temp2 = temp.drop(columns=['text','font_name','end','start','Y', 'Unnamed: 0', "Y_true"], axis = 1)
perm = PermutationImportance(model, random_state=44).fit(word, t["Y"])
eli5.show_weights(perm, feature_names = temp2.columns.tolist())

# в лучшем случае важность признаков:
# x_top_left    60%
# y_top_left    43%
# w2v           35%
# width         23%
# is_bold       23%
# is_normal     19%
# font_size      4%
# height         3%
# is_italic      0%

Weight,Feature
0.5231 ± 0.0802,x_top_left
0.3692 ± 0.0754,y_top_left
0.3292 ± 0.0314,w2v
0.1723 ± 0.0408,width
0.1600 ± 0.0718,is_bold
0.1477 ± 0.0314,is_normal
0.0308 ± 0.0000,height
0.0215 ± 0.0151,font_size
0 ± 0.0000,is_italic


In [18]:
import fitz
# Open the pdf
doc = fitz.open('C:/Users/Sheri/IdeaProjects/Work/temp_files/ticket10.pdf')
ttt = get_key_classes(get_name_keys(template))
for page in doc:
    page.wrap_contents()
    # For every page, draw a rectangle on coordinates (1,1)(100,100)
    for idx, row in temp.iterrows():
        for key, value in ttt.items():
            if value == row["Y"] & row["Y"] == row["Y_true"]:
                rect_x1 = row['x_top_left']
                rect_y1 = row["y_top_left"]
                rect = [rect_x1, rect_y1]
                page.insert_text([rect_x1, rect_y1-10], str(value), fontsize=14, color = (1, 0, 0))
                normalized_number1 = row["Y"] * 3 / 100
                normalized_number2 = row["Y"] * 2 / 100
                normalized_number3 = 1
                clr = (normalized_number1, normalized_number2, normalized_number3)
                page.draw_rect([rect_x1, rect_y1, rect_x1 + row["width"], rect_y1 + row["height"]],  color = clr, width = 2)
                # page.draw_rect([row['x_top_left'],row["y_top_left"],row['x_top_left'] + row["width"],row["y_top_left"] + row["height"]],  color = (1, 0 , 0), width = 2)

# Save pdf
doc.save('ticket11.pdf')


In [19]:
import fitz
# Open the pdf
doc = fitz.open('C:/Users/Sheri/IdeaProjects/Work/temp_files/ticket10.pdf')
ttt = get_key_classes(get_name_keys(template))
for page in doc:
    page.wrap_contents()
    # For every page, draw a rectangle on coordinates (1,1)(100,100)
    for idx, row in temp.iterrows():
        for key, value in ttt.items():
            if value == row["Y"] & row["Y"] != row["Y_true"]:
                rect_x1 = row['x_top_left']
                rect_y1 = row["y_top_left"]
                rect = [rect_x1, rect_y1]
                page.insert_text([rect_x1, rect_y1-10], str(value), fontsize=14, color = (1, 0, 0))
                normalized_number1 = 1
                normalized_number2 = row["Y"] * 2 / 100
                normalized_number3 = row["Y"] * 3 / 100
                clr = (normalized_number1, normalized_number2, normalized_number3)
                page.draw_rect([rect_x1, rect_y1, rect_x1 + row["width"], rect_y1 + row["height"]],  color = clr, width = 2)
                # page.draw_rect([row['x_top_left'],row["y_top_left"],row['x_top_left'] + row["width"],row["y_top_left"] + row["height"]],  color = (1, 0 , 0), width = 2)

# Save pdf
doc.save('ticket12.pdf')


In [20]:
import fitz
# Open the pdf
doc = fitz.open('C:/Users/Sheri/IdeaProjects/Work/temp_files/ticket10.pdf')
ttt = get_key_classes(get_name_keys(template))
for page in doc:
    page.wrap_contents()
    # For every page, draw a rectangle on coordinates (1,1)(100,100)
    for idx, row in temp.iterrows():
        for key, value in ttt.items():
            if value == row["Y"] & row["Y"] == row["Y_true"]:
                rect_x1 = row['x_top_left']
                rect_y1 = row["y_top_left"]
                rect = [rect_x1, rect_y1]
                page.insert_text([rect_x1, rect_y1-10], str(value), fontsize=14, color = (1, 0, 0))
                normalized_number1 = row["Y"] * 3 / 100
                normalized_number2 = row["Y"] * 2 / 100
                normalized_number3 = 1
                clr = (normalized_number1, normalized_number2, normalized_number3)
                page.draw_rect([rect_x1, rect_y1, rect_x1 + row["width"], rect_y1 + row["height"]],  color = clr, width = 2)
            elif value == row["Y"] & row["Y"] != row["Y_true"]:
                rect_x1 = row['x_top_left']
                rect_y1 = row["y_top_left"]
                rect = [rect_x1, rect_y1]
                page.insert_text([rect_x1, rect_y1-10], str(value), fontsize=14, color = (1, 0, 0))
                normalized_number1 = 1
                normalized_number2 = row["Y"] * 2 / 100
                normalized_number3 = row["Y"] * 3 / 100
                clr = (normalized_number1, normalized_number2, normalized_number3)
                page.draw_rect([rect_x1, rect_y1, rect_x1 + row["width"], rect_y1 + row["height"]],  color = clr, width = 2)
                # page.draw_rect([row['x_top_left'],row["y_top_left"],row['x_top_left'] + row["width"],row["y_top_left"] + row["height"]],  color = (1, 0 , 0), width = 2)

# Save pdf
doc.save('ticket13.pdf')